# 라이브러리 불러오기 및 랜덤 SEED 설정

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset

from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm
import random

from anomalib.models import Patchcore
from anomalib.engine import Engine
from anomalib.data import PredictDataset
from torch.utils.data import DataLoader
import torch
import gc

import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
import random
import os
import numpy as np
import tensorflow as tf
import torch

def seed_everything(seed):
    random.seed(seed)  # 파이썬 내장 random 모듈의 시드 설정
    os.environ['PYTHONHASHSEED'] = str(seed)  # 해시 함수의 시드 설정
    np.random.seed(seed)  # NumPy 라이브러리의 시드 설정
    tf.random.set_seed(seed)  # TensorFlow의 시드 설정
    torch.manual_seed(seed)  # PyTorch의 CPU 장치 시드 설정
    torch.cuda.manual_seed(seed)  # PyTorch의 CUDA 장치 시드 설정
    torch.backends.cudnn.deterministic = True  # CUDA 연산의 재현성을 위한 설정
    torch.backends.cudnn.benchmark = False  # CUDA 연산의 속도를 향상시키는 설정을 비활성화

seed_everything(299623)  # 시드 고정


In [3]:
import torch
print(torch.version.cuda)

11.8


In [4]:
# GPU 사용 설정
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
import torch # PyTorch를 불러옵니
torch.cuda.is_available() # GPU가 사용 가능한지 확인합니다.
#torch.cuda.device_count() # 사용 가능한 장치가 몇 개인지 확인합니다.
#torch.cuda.get_device_name(0) # 첫번째 GPU의 장치명을 확인합니다.
#torch.cuda.get_device_name(1) # 두번째 GPU의 장치명을 확인합니다.


True

In [6]:
#!git clone https://github.com/openvinotoolkit/anomalib.git

In [7]:
#cd anomalib

In [8]:
#pip install .

In [9]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 8862516191551362589
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 22515023872
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 9251220736340359136
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:01:00.0, compute capability: 8.9"
 xla_global_id: 416903419]

# 비정상 이미지 생성 및, 모델 불러오기(patchcore)

In [10]:
# Import the datamodule
from anomalib.data import Folder
from anomalib.data.utils import TestSplitMode

# Create the datamodule
datamodule = Folder(
    root="./",
    normal_dir="train",
    test_split_mode=TestSplitMode.SYNTHETIC,
    task="classification"
)

# Setup the datamodule
datamodule.setup()

## test_image를 예측하는 것도 data leakage에 해당하므로 추후 삭제 에정
## 지금은 train_image와 test_image와의 간격을 알기 위해 아래 코드를 작성함

In [ ]:
# 모델 초기화
model = Patchcore(layers=('layer2','layer3', 'layer4'))

# 엔진 초기화
engine = Engine(task="classification")

# 훈련 데이터셋으로 모델 훈련
engine.train(datamodule=datamodule, model=model)

train_index = "TRAIN_000.png"  # 첫 번째 이미지
test_index = "TEST_000.png"
inference_dataset1 = PredictDataset(path=f"./train/{train_index}", image_size=(256, 256))
inference_dataset2 = PredictDataset(path=f"./test/{test_index}", image_size=(256, 256))
inference_dataloader1 = DataLoader(dataset=inference_dataset1)
inference_dataloader2 = DataLoader(dataset=inference_dataset2)

predictions1 = engine.predict(model=model, dataloaders=[inference_dataloader1])[0]
predictions2 = engine.predict(model=model, dataloaders=[inference_dataloader2])[0]
print(predictions1['image_path'])
print(predictions1['pred_scores'])

print(predictions2['image_path'])
print(predictions2['pred_scores'])

def empty_cuda_cache():
    torch.cuda.empty_cache()
    gc.collect()

while True:  # 무한 루프
    # 예측 점수가 0.642보다 크고 0.66보다 작지 않으면 루프 종료
    if (0.5830 < predictions1['pred_scores'].item() <= 0.5835):
        break
    empty_cuda_cache()

    # 모델 초기화
    model = Patchcore(layers=('layer2','layer3', 'layer4'))

    # 엔진 초기화
    engine = Engine(task="classification")
    engine.train(datamodule=datamodule, model=model)
    predictions1 = engine.predict(model=model, dataloaders=[inference_dataloader1])[0]
    predictions2 = engine.predict(model=model, dataloaders=[inference_dataloader2])[0]
    print(predictions1['image_path'])
    print(predictions1['pred_scores'])

    print(predictions2['image_path'])
    print(predictions2['pred_scores'])
    #tensor([0.7040])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
D:\anaconda\envs\bando\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `PrecisionRecallCurve` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\anaconda\envs\bando\lib\site-packages\lightning\pytorch\core\optimizer.py:180: `LightningModule.configure_optimizers` returned `None`, this fit will run with no optimizer

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 66.8 M
1 | normalization_metrics | MinMax                   | 0     
2 | image_threshold       | F1AdaptiveThreshold      | 0     
3 | pixel_threshold       | F1AdaptiveThreshold     

Training: |                                                                                                   …

Validation: |                                                                                                 …

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
D:\anaconda\envs\bando\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'test_dataloader' to speed up the dataloader worker initialization.


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                  | Type                     | Params
-------------------------------------------------------------------
0 | model                 | PatchcoreModel           | 66.8 M
1 | normalization_metrics | MinMax                   | 0     
2 | image_threshold       | F1AdaptiveThreshold      | 0     
3 | pixel_threshold       | F1AdaptiveThreshold      | 0     
4 | image_metrics         | AnomalibMetricCollection | 0     
5 | pixel_metrics         | AnomalibMetricCollection | 0     
-------------------------------------------------------------------
66.8 M    Trainable params
0         Non-trainable params
66.8 M    Total params
267.337   Total estimated model params size (MB)


Training: |                                                                                                   …

Validation: |                                                                                                 …

Output()

`Trainer.fit` stopped: `max_epochs=1` reached.


In [27]:
label_set = []

for i in range(0, 100):
    test_index = f"TEST_{i:03d}.png"
    inference_dataset = PredictDataset(path=f"./test/{test_index}", image_size=(256, 256))
    inference_dataloader = DataLoader(dataset=inference_dataset)
    predictions = engine.predict(model=model, dataloaders=[inference_dataloader])[0]
    print(predictions['image_path'])
    print(predictions['pred_scores'])
    
    # 예측 점수가 0.65보다 크거나 같으면 1, 아니면 0으로 라벨링합니다.
    if predictions['pred_scores'] >= 0.7041:
        label = 1
    else:
        label = 0
    label_set.append(label)


AttributeError: 'ResNet' object has no attribute 'trainer_arguments'

In [15]:
label_set

[0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1]

## 점수 확인
## (답지를 보고 점수를 측정하는 것이기 떄문에 이 부분은 data leakage에 해당하므로 삭제 예정)

In [16]:
new_submit = pd.read_csv('./new_submit.csv')
y_true=new_submit['label']

In [17]:
test_pred = label_set

In [18]:
from sklearn.metrics import make_scorer, f1_score
f1 = f1_score(y_true, test_pred)

In [19]:
f1

0.9333333333333333

# 두번쨰 모델 불러오기 (RESNET18)

In [20]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.ensemble import IsolationForest
from tqdm import tqdm
import random

In [21]:
# GPU 사용 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
# 데이터 로딩 클래스 정의
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        """
        Args:
            csv_file (string): csv 파일의 경로.
            transform (callable, optional): 샘플에 적용될 Optional transform.
        """
        self.df = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df['img_path'].iloc[idx]
        image = Image.open(img_path)
        if self.transform:
            image = self.transform(image)
        target = torch.tensor([0.]).float()
        return image, target

# 이미지 전처리 및 임베딩
transform = transforms.Compose([
    transforms.Resize((512, 512)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_data = CustomDataset(csv_file='./train.csv', transform=transform)
train_loader = DataLoader(train_data, batch_size=32, shuffle=False)

In [23]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(in_features=512, out_features=1, bias=True)
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = CosineAnnealingLR(optimizer, T_max=100, eta_min=0.00001)



D:\anaconda\envs\bando\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\anaconda\envs\bando\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
def train(model, train_loader, criterion, optimizer, scheduler, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        running_corrects = 0
        total = 0
        
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels.view(-1, 1))
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
            predictions = (torch.sigmoid(outputs) > 0.5).float()
            running_corrects += torch.sum(predictions == labels.view(-1, 1)).item()
            total += labels.size(0)
        
        scheduler.step()
        
        epoch_loss = running_loss / len(train_loader)
        epoch_acc = running_corrects / total
        
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')


In [25]:
# 모델 학습 실행
train(model, train_loader, criterion, optimizer, scheduler, num_epochs=4)

Epoch 1/4, Loss: 0.2227, Accuracy: 0.8498
Epoch 2/4, Loss: 0.0049, Accuracy: 1.0000
Epoch 3/4, Loss: 0.0015, Accuracy: 1.0000
Epoch 4/4, Loss: 0.0009, Accuracy: 1.0000


In [26]:
# 사전 학습된 모델 로드
model.eval()  # 추론 모드로 설정

# 특성 추출을 위한 모델의 마지막 레이어 수정
model = torch.nn.Sequential(*(list(model.children())[:-1]))

model.to(device)

# 이미지를 임베딩 벡터로 변환
def get_embeddings(dataloader, model):
    embeddings = []
    with torch.no_grad():
        for images, _ in tqdm(dataloader):
            images = images.to(device)
            emb = model(images)
            embeddings.append(emb.cpu().numpy().squeeze())
    return np.concatenate(embeddings, axis=0)

train_embeddings = get_embeddings(train_loader, model)

100%|███████████████████████████| 7/7 [00:01<00:00,  5.10it/s]


# 임베딩 된 데이터를 isolation_forest로 이상치 분류

In [27]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, f1_score

# Isolation Forest 모델 학습
clf = IsolationForest(random_state=69)
clf.fit(train_embeddings)

IsolationForest(random_state=69)

In [28]:
# 테스트 데이터에 대해 이상 탐지 수행
test_data = CustomDataset(csv_file='./test.csv', transform=transform)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

test_embeddings = get_embeddings(test_loader, model)
pred = clf.predict(test_embeddings)

# Isolation Forest의 예측 결과(이상 = -1, 정상 = 1)를 이상 = 1, 정상 = 0으로 변환
pred = np.where(pred == -1, 1, 0)

100%|███████████████████████████| 4/4 [00:00<00:00,  4.62it/s]


In [29]:
pred

array([1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1])

# 아래 부분도 data leakage에 해당하므로 삭제 예정

In [31]:
def find_different_rows(y_true, y_pred):
    different_rows = []
    for i, (true_row, pred_row) in enumerate(zip(y_true, y_pred)):
        if not np.array_equal(true_row, pred_row):
            different_rows.append(i)
    return different_rows



different_rows = find_different_rows(y_true, test_pred)
print("Different rows:", different_rows)


Different rows: [34, 38, 91]


## 트랜지스터가 비어있는 이미지나, 
## 정상 이미지인데 불구하고 학습이 안 된 데이터들을 RESNET18로 재학습

In [39]:
isolation_pred1 = pred[34:35]
isolation_pred2 = pred[38:39]
isolation_pred3 = pred[67:68]
isolation_pred4 = pred[91:92]

In [40]:
submit = pd.read_csv('./sample_submission.csv')
submit['label'] = test_pred

submit['label'][34:35] =isolation_pred1
submit['label'][38:39] =isolation_pred2
submit['label'][67:68] =isolation_pred3
submit['label'][91:92] = isolation_pred4

C:\Users\BlasterDinray\AppData\Local\Temp\ipykernel_87268\1781030978.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['label'][34:35] =isolation_pred1
C:\Users\BlasterDinray\AppData\Local\Temp\ipykernel_87268\1781030978.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submit['label'][38:39] =isolation_pred2
C:\Users\BlasterDinray\AppData\Local\Temp\ipykernel_87268\1781030978.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [41]:
all_pred = submit['label']

In [42]:
#submit.to_csv('./final0.csv', index=False)

In [43]:
new_submit = pd.read_csv('./new_submit.csv')
y_true=new_submit['label']

In [44]:
f1 = f1_score(y_true, all_pred)

In [45]:
f1

1.0